## Imports


In [25]:
# Provides ways to work with large multidimensional arrays
import numpy as np

# Allows for further data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt  # Plotting
import matplotlib.dates as mdates  # Styling dates

import datetime as dt

import time

import yfinance as yf

# Used for file handling like deleting files
# and get data from a directory
import os
from os import listdir
from os.path import isfile, join

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
# Use plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

import warnings

warnings.simplefilter("ignore")


## Constants


In [26]:
# Location of files
PATH = "C:\\Users\\Eduardo Nuñez\\OneDrive\\Programming\\Python\\Derek Banas\\Investing\\AFP\\"
FILE_PATH = PATH + "DocumentoValorCuota.xlsx"
DATA_PATH = PATH + "AFP Data\\"

# Fund names
FUND_NAMES = ["A", "B", "C", "D", "E"]

# Bollinger Bands
# Number of days to evaluate the moving average of the bollinger bands
n_days = 20


## Import Data from file


### Read Data from Excel file


In [27]:
def get_df_from_excel(path):
    try:
        df = pd.read_excel(path, index_col=0)
    except FileNotFoundError as e:
        print(e)
    else:
        return df


### Create a dataframe for each Fund


In [28]:
def get_fund_df(df: pd.DataFrame, fund_name):
    """Return a sub-dataframe of the form (date,fund_x)"""
    return df.filter(["Día", fund_name], axis=1)


In [29]:
def get_fund_df_list(df: pd.DataFrame):
    """Return a list of dataframes of the form (index,column_x)"""
    funds_df_list = []
    for col in df.columns:
        funds_df_list.append(get_fund_df(df, col))
    return funds_df_list


## Add Statistics


### Add returns


In [30]:
def add_daily_return_to_df(df: pd.DataFrame, col):
    """Return df with a new column 'daily_return'"""
    df["daily_return"] = (df[col] / df[col].shift(1)) - 1
    return df


In [31]:
def add_cum_return_to_df(df: pd.DataFrame):
    """Return df with a new column 'cum_return'"""
    df["cum_return"] = (1 + df["daily_return"]).cumprod()
    return df


### Add Bollinger Bands

Bollinger Bands plot 2 lines using a moving average and the standard deviation defines how far apart the lines are. They also are used to define if prices are too high or low. When bands tighten it is believed a sharp price move in some direction. Prices tend to bounce off of the bands which provides potential market actions.

A strong trend should be noted if the price moves outside the band. If prices go over the resistance line it is in overbought territory and if it breaks through support it is a sign of an oversold position.


In [32]:
def add_bollinger_bands(df: pd.DataFrame, col, n_days):
    # the value 1.96 is the value for which P(X<1.96) = 0.975 and is commonly used
    # to evaluate confidence intervals
    window = df[col].rolling(window=n_days)

    df["middle_band"] = window.mean()
    df["upper_band"] = df["middle_band"] + 1.96 * window.std()
    df["lower_band"] = df["middle_band"] - 1.96 * window.std()
    return df


### Add Ichimoku Data to Dataframe

This ichimoku is adapted to the afp data which is just a single value per day


In [33]:
def add_ichimoku(df: pd.DataFrame, col):
    # Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
    hi_val = df[col].rolling(window=9).max()
    lo_val = df[col].rolling(window=9).min()
    df["Conversion"] = (hi_val + lo_val) / 2

    # Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
    hi_val2 = df[col].rolling(window=26).max()
    lo_val2 = df[col].rolling(window=26).min()
    df["Baseline"] = (hi_val2 + lo_val2) / 2

    # Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
    df["SpanA"] = ((df["Conversion"] + df["Baseline"]) / 2).shift(26)

    # Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)
    hi_val3 = df[col].rolling(window=52).max()
    lo_val3 = df[col].rolling(window=52).min()
    df["SpanB"] = ((hi_val3 + lo_val3) / 2).shift(26)

    # Lagging Span = Price shifted back 26 periods
    df["Lagging"] = df[col].shift(-26)

    return df


### Adding MACD values


In [34]:
# Calculating EMA
def add_EMA(df: pd.DataFrame, col, n_periods=12):
    sma = df[col].rolling(window=n_periods).mean()
    mult = 2 / (13)
    ema = sma
    ema = (df[col] - ema.shift(1)) * mult + ema.shift(1)
    df[f"sma_{n_periods}"] = sma
    df[f"ema_{n_periods}"] = ema
    return df


In [35]:
# Calculating MACD
def add_MACD(df: pd.DataFrame, col):
    # Adding EMA 12 and 26
    df = add_EMA(df, col, n_periods=12)
    df = add_EMA(df, col, n_periods=26)
    # Adding the MACD
    df["MACD"] = df["ema_12"] - df["ema_26"]
    return df


## Add Daily, Cumulative Bollinger Bands, Ichimoku and MACD to Dataframes


In [36]:
df = get_df_from_excel(FILE_PATH)
df.sort_index(inplace=True)
fund_df_list = get_fund_df_list(df)
for fund_df in fund_df_list:
    fund_name = fund_df.columns[0]
    fund_df = add_daily_return_to_df(fund_df, fund_name)
    fund_df = add_cum_return_to_df(fund_df)
    fund_df = add_bollinger_bands(fund_df, fund_name, n_days)
    fund_df = add_ichimoku(fund_df, fund_name)
    fund_df = add_MACD(fund_df, fund_name)
    fund_df.to_csv(DATA_PATH + fund_name + ".csv")

print("Finished")


Finished
